Analysis of the attack models data in order to see why they are majority classifiers

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from pathlib import Path
import os
import sys
import bz2
import pickle
np.random.seed(10)
# adds the visibility of the mlem module, needed to load the attack models
sys.path.append("../../../../..") 
import mlem
from mlem.ensemble import HardVotingClassifier
from mlem.utilities import report_and_confusion, load_pickle_bz2

In [2]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

In [3]:
BLACK_BOX_PATH = '../BB_DATA/adult_rf_noclustering.bz2'
black_box = load_pickle_bz2(BLACK_BOX_PATH)

In [4]:
BASE_BB_DATA_PATH = "../BB_DATA/adult_rf_noclustering_data_nsamples_{n_samples}.npz"
loaded = np.load(BASE_BB_DATA_PATH.format(n_samples=1), allow_pickle=True)
for k in loaded.keys():
    print(k, end=" ")
x_train = loaded["x_train"]
x_test = loaded["x_test"]
y_train = loaded["y_train"]
y_test = loaded["y_test"]
x_test_clustered = loaded["x_test_clustered"]
y_test_clustered = loaded["y_test_clustered"]

x_train x_test y_train y_test x_test_clustered y_test_clustered 

# Load Attack Models and data

In [5]:
def load_atk_and_data(idx, base_path="/home/gerardozinno/Desktop/ExperimentsResult/ADA/ADULT/adult_results_noclust_ada_1/gaussian/same/"):
    index_template = Path(base_path) / f"{idx}" / "attack"
    atk_0 = load_pickle_bz2(index_template / "0" / "model.pkl.bz2")
    atk_1 = load_pickle_bz2(index_template / "1" / "model.pkl.bz2")
    data_0 = np.load(index_template / "0" / "data.npz" ,allow_pickle=True)
    data_1 = np.load(index_template / "1" / "data.npz" ,allow_pickle=True)
    return (atk_0, data_0), (atk_1, data_1)

## idx 0

In [6]:
(atk_0, data_0), (atk_1, data_1) = load_atk_and_data(0)
for k in data_0.keys():
    print(k, end=" ")

x_train x_test y_train y_test 

/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### train

Trainset for the attack model trained to recognise the label 0

In [7]:
d0_train = pd.DataFrame(data_0['x_train'])
d0_train['Target'] = data_0['y_train']

d0_train.value_counts()

0         1         Target
0.535312  0.464688  in        52
0.544175  0.455825  in        45
0.533603  0.466397  in        31
0.535312  0.464688  out       20
0.553686  0.446314  in        20
                              ..
0.537402  0.462598  out        1
0.537403  0.462597  in         1
0.537410  0.462590  in         1
0.537413  0.462587  in         1
0.741065  0.258935  in         1
Length: 13786, dtype: int64

This values appear with two different labels:

    0.535312  0.464688  in        52
    0.535312  0.464688  out       20

In [8]:
d0_train.Target.value_counts(normalize=True)

in     0.802888
out    0.197112
Name: Target, dtype: float64

In [9]:
d0_train.describe()

,0,1
count,15996.000000,15996.000000
mean,0.549075,0.450925
std,0.025819,0.025819
min,0.490693,0.258935
25%,0.532639,0.438618
50%,0.545928,0.454072
75%,0.561382,0.467361
max,0.741065,0.509307


Trainset for the attack model trained to recognise the label 1

In [10]:
d1_train = pd.DataFrame(data_1['x_train'])
d1_train['Target'] = data_1['y_train']
d1_train.value_counts()

0         1         Target
0.414062  0.585938  in        23
0.457965  0.542035  in        19
0.489567  0.510433  in        19
0.353937  0.646063  in        18
0.489991  0.510009  in        16
                              ..
0.439280  0.560720  in         1
0.439235  0.560765  in         1
0.439227  0.560773  in         1
0.439155  0.560845  in         1
0.506060  0.493940  out        1
Length: 9487, dtype: int64

The out class is not represented enough

In [11]:
d1_train.Target.value_counts(normalize=True)

in     0.96737
out    0.03263
Name: Target, dtype: float64

In [12]:
d1_train.describe()

,0,1
count,13270.000000,13270.000000
mean,0.440449,0.559551
std,0.040985,0.040985
min,0.277925,0.493940
25%,0.409709,0.524964
50%,0.451793,0.548207
75%,0.475036,0.590291
max,0.506060,0.722075


#### test

Testset for the attack model trained to recognise the label 0

In [13]:
d0_test = pd.DataFrame(data_0['x_test'])
d0_test['Target'] = data_0['y_test']
d0_test.value_counts()

0         1         Target
0.535312  0.464688  in        17
0.533603  0.466397  in         9
0.533771  0.466229  in         9
0.544175  0.455825  in         9
0.547973  0.452027  in         9
                              ..
0.537127  0.462873  in         1
0.537112  0.462888  out        1
0.537088  0.462912  in         1
0.537084  0.462916  in         1
0.709064  0.290936  in         1
Length: 4925, dtype: int64

In [14]:
d0_test.Target.value_counts(normalize=True)

in     0.7982
out    0.2018
Name: Target, dtype: float64

In [15]:
d0_test.describe()

,0,1
count,5332.000000,5332.000000
mean,0.549161,0.450839
std,0.026303,0.026303
min,0.451728,0.290936
25%,0.532076,0.438898
50%,0.545957,0.454043
75%,0.561102,0.467924
max,0.709064,0.548272


Testset for the attack model trained to recognise the label 1

In [16]:
d1_test = pd.DataFrame(data_1['x_test'])
d1_test['Target'] = data_1['y_test']
d1_test.value_counts()

0         1         Target
0.417448  0.582552  in        9
0.457965  0.542035  in        8
0.489991  0.510009  in        8
0.440280  0.559720  in        7
0.469984  0.530016  in        6
                             ..
0.432105  0.567895  in        1
0.432172  0.567828  in        1
0.432228  0.567772  in        1
0.432369  0.567631  in        1
0.502548  0.497452  out       1
Length: 3797, dtype: int64

The out class is not represented enough

In [17]:
d1_test.Target.value_counts(normalize=True)

in     0.963382
out    0.036618
Name: Target, dtype: float64

In [18]:
d1_test.describe()

,0,1
count,4424.000000,4424.000000
mean,0.440667,0.559333
std,0.040722,0.040722
min,0.293519,0.497452
25%,0.411173,0.525503
50%,0.451985,0.548015
75%,0.474497,0.588827
max,0.502548,0.706481


## idx 1

In [19]:
(atk_0, data_0), (atk_1, data_1) = load_atk_and_data(1)
for k in data_0.keys():
    print(k, end=" ")

x_train x_test y_train y_test 

/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### train

Trainset for the attack model trained to recognise the label 0

In [20]:
d0_train = pd.DataFrame(data_0['x_train'])
d0_train['Target'] = data_0['y_train']

d0_train.value_counts()

0         1         Target
0.546289  0.453711  in        16
0.513124  0.486876  in        15
0.606937  0.393063  in        14
0.513884  0.486116  in        12
0.608273  0.391727  in        12
                              ..
0.539924  0.460076  in         1
0.539922  0.460078  in         1
0.539914  0.460086  in         1
0.539912  0.460088  out        1
0.656645  0.343355  in         1
Length: 9199, dtype: int64

Class out misrepresented

In [21]:
d0_train.Target.value_counts(normalize=True)

in     0.978845
out    0.021155
Name: Target, dtype: float64

In [22]:
d0_train.describe()

,0,1
count,13614.000000,13614.000000
mean,0.563660,0.436340
std,0.038137,0.038137
min,0.460994,0.343355
25%,0.528731,0.399625
50%,0.555570,0.444430
75%,0.600375,0.471269
max,0.656645,0.539006


Trainset for the attack model trained to recognise the label 1

In [23]:
d1_train = pd.DataFrame(data_1['x_train'])
d1_train['Target'] = data_1['y_train']
d1_train.value_counts()

0         1         Target
0.444086  0.555914  in        14
0.440188  0.559812  in        10
0.438133  0.561867  in         9
0.442315  0.557685  in         9
0.445389  0.554611  in         9
                              ..
0.436329  0.563671  in         1
                    out        1
0.436330  0.563670  in         1
0.436335  0.563665  in         1
0.526595  0.473405  out        1
Length: 15717, dtype: int64

The out class is not represented enough

In [24]:
d1_train.Target.value_counts(normalize=True)

in     0.798835
out    0.201165
Name: Target, dtype: float64

In [25]:
d1_train.describe()

,0,1
count,16653.000000,16653.000000
mean,0.442087,0.557913
std,0.023782,0.023782
min,0.296897,0.473405
25%,0.430463,0.543177
50%,0.443774,0.556226
75%,0.456823,0.569537
max,0.526595,0.703103


#### test

Testset for the attack model trained to recognise the label 0

In [26]:
d0_test = pd.DataFrame(data_0['x_test'])
d0_test['Target'] = data_0['y_test']
d0_test.value_counts()

0         1         Target
0.608273  0.391727  in        9
0.546289  0.453711  in        9
0.512205  0.487795  in        6
0.606937  0.393063  in        6
0.602440  0.397560  in        6
                             ..
0.537026  0.462974  in        1
0.537039  0.462961  in        1
0.537084  0.462916  in        1
0.537135  0.462865  in        1
0.660830  0.339170  in        1
Length: 3844, dtype: int64

In [27]:
d0_test.Target.value_counts(normalize=True)

in     0.978405
out    0.021595
Name: Target, dtype: float64

In [28]:
d0_test.describe()

,0,1
count,4538.000000,4538.000000
mean,0.563852,0.436148
std,0.037960,0.037960
min,0.495058,0.339170
25%,0.529039,0.399434
50%,0.555730,0.444270
75%,0.600566,0.470961
max,0.660830,0.504942


Testset for the attack model trained to recognise the label 1

In [29]:
d1_test = pd.DataFrame(data_1['x_test'])
d1_test['Target'] = data_1['y_test']
d1_test.value_counts()

0         1         Target
0.439130  0.560870  in        6
0.444536  0.555464  in        4
0.395306  0.604694  in        3
0.438133  0.561867  in        3
0.439071  0.560929  in        3
                             ..
0.435828  0.564172  out       1
0.435827  0.564173  out       1
                    in        1
0.435812  0.564188  in        1
0.515932  0.484068  out       1
Length: 5436, dtype: int64

The out class is not represented enough

In [30]:
d1_test.Target.value_counts(normalize=True)

in     0.803999
out    0.196001
Name: Target, dtype: float64

In [31]:
d1_test.describe()

,0,1
count,5551.000000,5551.000000
mean,0.442042,0.557958
std,0.023785,0.023785
min,0.325239,0.484068
25%,0.430324,0.543051
50%,0.444018,0.555982
75%,0.456949,0.569676
max,0.515932,0.674761


## idx 2

In [32]:
(atk_0, data_0), (atk_1, data_1) = load_atk_and_data(2)
for k in data_0.keys():
    print(k, end=" ")

x_train x_test y_train y_test 

/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### train

Trainset for the attack model trained to recognise the label 0

In [33]:
d0_train = pd.DataFrame(data_0['x_train'])
d0_train['Target'] = data_0['y_train']

d0_train.value_counts()

0         1         Target
0.532207  0.467793  in        15
0.613668  0.386332  in        13
0.541932  0.458068  in        11
0.536388  0.463612  in        11
0.526574  0.473426  in        11
                              ..
0.533975  0.466025  in         1
0.533971  0.466029  in         1
0.533956  0.466044  in         1
0.533950  0.466050  in         1
0.686169  0.313831  in         1
Length: 9630, dtype: int64

This values appear with two different labels:

    0.535312  0.464688  in        52
    0.535312  0.464688  out       20

In [34]:
d0_train.Target.value_counts(normalize=True)

in     0.977273
out    0.022727
Name: Target, dtype: float64

In [35]:
d0_train.describe()

,0,1
count,13464.000000,13464.000000
mean,0.557400,0.442600
std,0.035502,0.035502
min,0.490198,0.313831
25%,0.526980,0.409847
50%,0.545865,0.454135
75%,0.590153,0.473020
max,0.686169,0.509802


Trainset for the attack model trained to recognise the label 1

In [36]:
d1_train = pd.DataFrame(data_1['x_train'])
d1_train['Target'] = data_1['y_train']
d1_train.value_counts()

0         1         Target
0.455082  0.544918  in        6
0.455955  0.544045  in        5
0.455503  0.544497  in        5
0.459932  0.540068  in        5
0.456697  0.543303  in        5
                             ..
0.442260  0.557740  in        1
0.442267  0.557733  in        1
0.442270  0.557730  in        1
0.442274  0.557726  out       1
0.512038  0.487962  out       1
Length: 15824, dtype: int64

The out class is not represented enough

In [37]:
d1_train.Target.value_counts(normalize=True)

in     0.797647
out    0.202353
Name: Target, dtype: float64

In [38]:
d1_train.describe()

,0,1
count,16491.000000,16491.000000
mean,0.446852,0.553148
std,0.023001,0.023001
min,0.329721,0.487962
25%,0.436922,0.538917
50%,0.449712,0.550288
75%,0.461083,0.563078
max,0.512038,0.670279


#### test

Testset for the attack model trained to recognise the label 0

In [39]:
d0_test = pd.DataFrame(data_0['x_test'])
d0_test['Target'] = data_0['y_test']
d0_test.value_counts()

0         1         Target
0.574948  0.425052  in        5
0.529523  0.470477  in        5
0.586836  0.413164  in        4
0.601012  0.398988  in        4
0.590132  0.409868  in        4
                             ..
0.533057  0.466943  in        1
0.533083  0.466917  in        1
0.533130  0.466870  in        1
0.533144  0.466856  in        1
0.677628  0.322372  in        1
Length: 3916, dtype: int64

In [40]:
d0_test.Target.value_counts(normalize=True)

in     0.980392
out    0.019608
Name: Target, dtype: float64

In [41]:
d0_test.describe()

,0,1
count,4488.000000,4488.000000
mean,0.557191,0.442809
std,0.035552,0.035552
min,0.499355,0.322372
25%,0.526784,0.409824
50%,0.545070,0.454930
75%,0.590176,0.473216
max,0.677628,0.500645


Testset for the attack model trained to recognise the label 1

In [42]:
d1_test = pd.DataFrame(data_1['x_test'])
d1_test['Target'] = data_1['y_test']
d1_test.value_counts()

0         1         Target
0.455955  0.544045  in        4
0.461945  0.538055  in        3
0.413510  0.586490  in        3
0.430222  0.569778  in        3
0.437150  0.562850  in        3
                             ..
0.442170  0.557830  out       1
0.442160  0.557840  in        1
0.442156  0.557844  out       1
0.442145  0.557855  in        1
0.525878  0.474122  out       1
Length: 5396, dtype: int64

The out class is not represented enough

In [43]:
d1_test.Target.value_counts(normalize=True)

in     0.801164
out    0.198836
Name: Target, dtype: float64

In [44]:
d1_test.describe()

,0,1
count,5497.000000,5497.000000
mean,0.446982,0.553018
std,0.022745,0.022745
min,0.333267,0.474122
25%,0.436466,0.538522
50%,0.449751,0.550249
75%,0.461478,0.563534
max,0.525878,0.666733


## idx 20

In [45]:
(atk_0, data_0), (atk_1, data_1) = load_atk_and_data(20)
for k in data_0.keys():
    print(k, end=" ")

x_train x_test y_train y_test 

/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### train

Trainset for the attack model trained to recognise the label 0

In [46]:
d0_train = pd.DataFrame(data_0['x_train'])
d0_train['Target'] = data_0['y_train']

d0_train.value_counts()

0         1         Target
0.526360  0.473640  in        13
0.530328  0.469672  in        12
0.525273  0.474727  in        12
0.616675  0.383325  in        11
0.522925  0.477075  in        10
                              ..
0.534068  0.465932  in         1
0.534063  0.465937  in         1
0.534054  0.465946  in         1
0.534051  0.465949  in         1
0.703273  0.296727  out        1
Length: 9427, dtype: int64

In [47]:
d0_train.Target.value_counts(normalize=True)

in     0.978093
out    0.021907
Name: Target, dtype: float64

In [48]:
d0_train.describe()

,0,1
count,13603.000000,13603.000000
mean,0.563816,0.436184
std,0.044036,0.044036
min,0.441062,0.296727
25%,0.525800,0.404067
50%,0.550138,0.449862
75%,0.595933,0.474200
max,0.703273,0.558938


Trainset for the attack model trained to recognise the label 1

In [49]:
d1_train = pd.DataFrame(data_1['x_train'])
d1_train['Target'] = data_1['y_train']
d1_train.value_counts()

0         1         Target
0.433837  0.566163  in        49
0.419578  0.580422  in        18
0.416365  0.583635  in        17
0.448298  0.551702  in        13
0.441793  0.558207  in        13
                              ..
0.438088  0.561912  in         1
0.438102  0.561898  out        1
0.438112  0.561888  in         1
0.438113  0.561887  out        1
0.547629  0.452371  out        1
Length: 15433, dtype: int64

The out class is not represented enough

In [50]:
d1_train.Target.value_counts(normalize=True)

in     0.799326
out    0.200674
Name: Target, dtype: float64

In [51]:
d1_train.describe()

,0,1
count,16614.000000,16614.000000
mean,0.442457,0.557543
std,0.028208,0.028208
min,0.232238,0.452371
25%,0.430219,0.539812
50%,0.446104,0.553896
75%,0.460188,0.569781
max,0.547629,0.767762


#### test

Testset for the attack model trained to recognise the label 0

In [52]:
d0_test = pd.DataFrame(data_0['x_test'])
d0_test['Target'] = data_0['y_test']
d0_test.value_counts()

0         1         Target
0.655192  0.344808  in        6
0.653809  0.346191  in        5
0.595572  0.404428  in        5
0.633207  0.366793  in        4
0.635057  0.364943  in        4
                             ..
0.533715  0.466285  in        1
0.533719  0.466281  in        1
0.533723  0.466277  in        1
0.533759  0.466241  in        1
0.701896  0.298104  in        1
Length: 3919, dtype: int64

In [53]:
d0_test.Target.value_counts(normalize=True)

in     0.976406
out    0.023594
Name: Target, dtype: float64

In [54]:
d0_test.describe()

,0,1
count,4535.000000,4535.000000
mean,0.563729,0.436271
std,0.043412,0.043412
min,0.477135,0.298104
25%,0.526371,0.404742
50%,0.551201,0.448799
75%,0.595258,0.473629
max,0.701896,0.522865


Testset for the attack model trained to recognise the label 1

In [55]:
d1_test = pd.DataFrame(data_1['x_test'])
d1_test['Target'] = data_1['y_test']
d1_test.value_counts()

0         1         Target
0.433837  0.566163  in        9
0.416365  0.583635  in        7
0.447169  0.552831  in        5
0.443509  0.556491  in        5
0.433837  0.566163  out       5
                             ..
0.437407  0.562593  in        1
0.437393  0.562607  out       1
0.437388  0.562612  in        1
0.437384  0.562616  in        1
0.561107  0.438893  out       1
Length: 5366, dtype: int64

The out class is not represented enough

In [56]:
d1_test.Target.value_counts(normalize=True)

in     0.804081
out    0.195919
Name: Target, dtype: float64

In [57]:
d1_test.describe()

,0,1
count,5538.000000,5538.000000
mean,0.442411,0.557589
std,0.029245,0.029245
min,0.266011,0.438893
25%,0.430392,0.539750
50%,0.446921,0.553079
75%,0.460250,0.569608
max,0.561107,0.733989


## idx 23

In [58]:
(atk_0, data_0), (atk_1, data_1) = load_atk_and_data(23)
for k in data_0.keys():
    print(k, end=" ")

x_train x_test y_train y_test 

/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/gerardozinno/.pyenv/versions/ml-environment/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator AdaBoostClassifier from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


#### train

Trainset for the attack model trained to recognise the label 0

In [59]:
d0_train = pd.DataFrame(data_0['x_train'])
d0_train['Target'] = data_0['y_train']

d0_train.value_counts()

0         1         Target
0.521133  0.478867  in        15
0.646191  0.353809  in        13
0.611000  0.389000  in        11
0.627770  0.372230  in        11
0.530208  0.469792  in        11
                              ..
0.530394  0.469606  in         1
0.530385  0.469615  in         1
0.530383  0.469617  in         1
0.530375  0.469625  in         1
0.539874  0.460126  in         1
Length: 9432, dtype: int64

In [60]:
d0_train.Target.value_counts(normalize=True)

in     0.976233
out    0.023767
Name: Target, dtype: float64

In [61]:
d0_train.describe()

,0,1
count,13506.000000,13506.000000
mean,0.560074,0.439926
std,0.043507,0.043507
min,0.487665,0.310961
25%,0.523208,0.404788
50%,0.542158,0.457842
75%,0.595212,0.476792
max,0.689039,0.512335


Trainset for the attack model trained to recognise the label 1

In [62]:
d1_train = pd.DataFrame(data_1['x_train'])
d1_train['Target'] = data_1['y_train']
d1_train.value_counts()

0         1         Target
0.437436  0.562564  in        23
0.436723  0.563277  in        15
0.448443  0.551557  in        12
0.442516  0.557484  in        11
0.440409  0.559591  in        11
                              ..
0.445384  0.554616  in         1
0.445387  0.554613  in         1
0.445388  0.554612  in         1
0.445410  0.554590  out        1
0.549557  0.450443  out        1
Length: 14797, dtype: int64

The out class is not represented enough

In [63]:
d1_train.Target.value_counts(normalize=True)

in     0.801019
out    0.198981
Name: Target, dtype: float64

In [64]:
d1_train.describe()

,0,1
count,16479.000000,16479.000000
mean,0.450192,0.549808
std,0.023906,0.023906
min,0.299624,0.450443
25%,0.439139,0.535256
50%,0.452271,0.547729
75%,0.464744,0.560861
max,0.549557,0.700376


#### test

Testset for the attack model trained to recognise the label 0

In [65]:
d0_test = pd.DataFrame(data_0['x_test'])
d0_test['Target'] = data_0['y_test']
d0_test.value_counts()

0         1         Target
0.629463  0.370537  in        7
0.611000  0.389000  in        6
0.641376  0.358624  in        5
0.519981  0.480019  in        5
0.609630  0.390370  in        5
                             ..
0.529427  0.470573  in        1
0.529460  0.470540  in        1
                    in        1
0.529470  0.470530  in        1
0.689039  0.310961  in        1
Length: 3917, dtype: int64

In [66]:
d0_test.Target.value_counts(normalize=True)

in     0.975789
out    0.024211
Name: Target, dtype: float64

In [67]:
d0_test.describe()

,0,1
count,4502.000000,4502.000000
mean,0.560423,0.439577
std,0.043533,0.043533
min,0.493918,0.310961
25%,0.523560,0.402927
50%,0.542807,0.457193
75%,0.597073,0.476440
max,0.689039,0.506082


Testset for the attack model trained to recognise the label 1

In [68]:
d1_test = pd.DataFrame(data_1['x_test'])
d1_test['Target'] = data_1['y_test']
d1_test.value_counts()

0         1         Target
0.430092  0.569908  in        8
0.437436  0.562564  in        7
0.425165  0.574835  in        5
0.451819  0.548181  in        5
0.438605  0.561395  in        5
                             ..
0.445480  0.554520  in        1
0.445477  0.554523  in        1
0.445476  0.554524  out       1
0.445459  0.554541  out       1
0.503980  0.496020  out       1
Length: 5196, dtype: int64

The out class is not represented enough

In [69]:
d1_test.Target.value_counts(normalize=True)

in     0.797014
out    0.202986
Name: Target, dtype: float64

In [70]:
d1_test.describe()

,0,1
count,5493.000000,5493.000000
mean,0.450585,0.549415
std,0.023016,0.023016
min,0.309350,0.496020
25%,0.439252,0.535221
50%,0.453000,0.547000
75%,0.464779,0.560748
max,0.503980,0.690650
